In [1]:
from nltk import word_tokenize,sent_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import stopwords
import os
lemmatizer = WordNetLemmatizer()
punctuations = '''!()-[]{};—:'"\,<>./’?@#$%^&*_~“”'''
stop_words = stopwords.words('english') + list(punctuations)

In [3]:
lex_words=[]
f = open("SentiLex.txt","r",encoding='utf-8') 
lines=f.readlines() 
for line in lines:
    w=line.split('\t')[1]
    lex_words.append(w)

In [4]:
def convert_tag(tag):
    
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [5]:
def sent_words(text):
    for c in punctuations:
             text= text.replace(c," ")
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]

In [6]:
def calc_sent(text):
    sentences=sent_tokenize(text)
    sentiment=0.0
    for s in sentences:
        sentence_sentiment=0.0
        w=sent_words(s)
        tagged_words=pos_tag(w)
        #print(tagged_words)
        for word,tag in tagged_words:
            wn_tag = convert_tag(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
                continue
            lemma=lemmatizer.lemmatize(word,wn_tag)
            if word in lex_words:
                line_no=lines[lex_words.index(word)]
                pos_score,neg_score=float(line_no.split('\t')[2]),float(line_no.split('\t')[3])
            #or else if lemma in sentilex
            
            elif lemma in lex_words: 
                line_no=lines[lex_words.index(lemma)]
                pos_score,neg_score=float(line_no.split('\t')[2]),float(line_no.split('\t')[3])
                #print(word,'-',lemma)
            #if not then skip
            else:
                continue
            sentence_sentiment += pos_score - neg_score
        sentiment+=sentence_sentiment
    return sentiment/len(sentences)

In [12]:
path="E:/ProjectS-master/Statesman_gst/July/Articles/"
l=os.listdir(path)
wdfile=open('Statesman_July_Domain.txt','a',encoding='utf-8')
wsfile=open('Statesman_July_SentiWordnet.txt','a',encoding='utf-8')
for i in l:
    try:
        file=open(path+i,"r",encoding='utf-8')
        article=file.read()
        sent_score_domain_lexicon=calc_sent(article)
        sent_score_sentiwordnet_lexicon=swn_sent(article)
        wdfile.write(str(i.replace('.txt',''))+'\t'+str(sent_score_domain_lexicon)+'\n')
        wsfile.write(str(i.replace('.txt',''))+'\t'+str(sent_score_sentiwordnet_lexicon)+'\n')
    except:
        print(i)
wdfile.close()
wsfile.close()

In [8]:
def swn_sent(text):
    sentences=sent_tokenize(text)
    sentiment=0.0
    for s in sentences:
        sentence_sentiment=0.0
        w=sent_words(s)
        tagged_words=pos_tag(w)
        #print(tagged_words)
        for word,tag in tagged_words:
            wn_tag = convert_tag(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV,wn.VERB):
                continue
            lemma=lemmatizer.lemmatize(word,wn_tag)
            if not lemma:
                continue
 
            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue
 
            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())

            sentence_sentiment += swn_synset.pos_score() - swn_synset.neg_score()
        sentiment+=sentence_sentiment
    return sentiment/len(sentences)
       

   
    